# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = pd.read_csv("../output_data/cities.csv")


output_data_file


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.6001,-73.9662,60.53,83.0,34.0,2.53,US,1.627796e+09
1,mataura,-46.1927,168.8643,48.47,99.0,100.0,1.48,NZ,1.627796e+09
2,rikitea,-23.1203,-134.9692,70.47,72.0,44.0,19.57,PF,1.627796e+09
3,namatanai,-3.6667,152.4333,82.92,70.0,29.0,3.47,PG,1.627796e+09
4,labuhan,-6.8844,112.2051,87.80,56.0,14.0,6.51,ID,1.627796e+09
...,...,...,...,...,...,...,...,...,...
607,cam pha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,utiroa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,mackay,-21.1500,149.2000,81.68,61.0,33.0,14.97,AU,1.627797e+09
610,sinnamary,5.3833,-52.9500,77.70,86.0,91.0,6.71,GF,1.627797e+09


In [3]:
output_data_file.describe()

,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,5.550000e+02
mean,19.856064,20.151992,70.419441,68.576577,51.731532,7.589207,1.627796e+09
std,32.477412,91.361169,14.404284,21.924695,38.773502,5.482809,2.465835e+02
min,-54.800000,-175.200000,33.330000,7.000000,0.000000,0.000000,1.627796e+09
25%,-6.860550,-63.231700,59.740000,55.000000,10.500000,3.440000,1.627796e+09
50%,25.519800,25.570100,72.270000,73.000000,53.000000,6.400000,1.627796e+09
75%,44.832900,102.015050,82.050000,86.000000,91.000000,10.360000,1.627797e+09
max,78.218600,179.316700,104.490000,100.000000,100.000000,31.070000,1.627797e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [38]:
# Store 'Lat' and 'Lng' into  locations 
location = output_data_file[["Latitude", "Longitude"]].astype(float)
location_complete = location.dropna()

humidity = output_data_file["Humidity"].astype(float)
humidity_complete = humidity.dropna()


In [54]:
# Create a poverty Heatmap layer
fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(location_complete, weights=humidity_complete,
                                 max_intensity=100,
                                 point_radius = 10)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
#null is already dropped from the previous step
#creating dataframe

ideal_weather = output_data_file.loc[(output_data_file["Max Temp"] >= 70) & (output_data_file["Max Temp"] <= 80) & (output_data_file["Wind Speed"] < 10) & (output_data_file["Cloudiness"] == 0)].dropna()
ideal_weather

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,ukiah,39.1502,-123.2078,74.01,59.0,0.0,0.60,US,1.627796e+09
67,east london,-33.0153,27.9116,70.74,16.0,0.0,6.91,ZA,1.627796e+09
85,saint-pierre,-21.3393,55.4781,74.88,54.0,0.0,0.00,RE,1.627796e+09
319,changji,44.0167,87.3167,77.79,36.0,0.0,4.47,CN,1.627796e+09
334,turayf,31.6725,38.6637,77.25,41.0,0.0,6.91,SA,1.627796e+09
337,zarkos,39.6167,22.1333,70.00,64.0,0.0,0.00,GR,1.627796e+09
397,asau,46.4333,26.4000,77.40,56.0,0.0,2.51,RO,1.627797e+09
431,elbistan,38.2059,37.1983,74.17,30.0,0.0,2.77,TR,1.627797e+09
531,urumqi,43.8010,87.6005,76.15,36.0,0.0,4.47,CN,1.627797e+09
572,malatya,38.5000,38.0000,77.05,34.0,0.0,4.61,TR,1.627797e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df = ideal_weather.loc[:,["City", "Country","Latitude","Longitude"]]

# Add hotel column to the df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
14,ukiah,US,39.1502,-123.2078,
67,east london,ZA,-33.0153,27.9116,
85,saint-pierre,RE,-21.3393,55.4781,
319,changji,CN,44.0167,87.3167,
334,turayf,SA,31.6725,38.6637,
337,zarkos,GR,39.6167,22.1333,
397,asau,RO,46.4333,26.4000,
431,elbistan,TR,38.2059,37.1983,
531,urumqi,CN,43.8010,87.6005,
572,malatya,TR,38.5000,38.0000,


In [42]:
#Set parameters to search for hotels with 5000 meters
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.


hotels = []

for indx, row in hotel_df.iterrows():
    lat = row["Latitude"]
    long = row["Longitude"]
    
    # Target target_coordinates
    target_coordinates = f"{lat}, {long}"
    target_search = "hotel"
    target_radius = 50000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "keyword": "hotel",
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    places_data = response.json()

    try:
        hotel = places_data["results"][0]["name"]

        #append to lists
        hotels.append(hotel)

    except:
        hotels.append("")
    
hotel_df["Hotel Name"] = hotels
hotel_df


,City,Country,Latitude,Longitude,Hotel Name
14,ukiah,US,39.1502,-123.2078,Wildflower Boutique Motel
67,east london,ZA,-33.0153,27.9116,Southern Sun Hemingways
85,saint-pierre,RE,-21.3393,55.4781,Lodge Roche Tamarin & Spa Village Nature
319,changji,CN,44.0167,87.3167,Hilton Urumqi
334,turayf,SA,31.6725,38.6637,Le Park Concord Turayf
337,zarkos,GR,39.6167,22.1333,Montanema Handmade Village
397,asau,RO,46.4333,26.4000,Balvanyos Resort
431,elbistan,TR,38.2059,37.1983,Grand Elbistan Park Hotel
531,urumqi,CN,43.8010,87.6005,Hilton Urumqi
572,malatya,TR,38.5000,38.0000,Movenpick Malatya Hotel


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
locations

,Latitude,Longitude
14,39.1502,-123.2078
67,-33.0153,27.9116
85,-21.3393,55.4781
319,44.0167,87.3167
334,31.6725,38.6637
337,39.6167,22.1333
397,46.4333,26.4000
431,38.2059,37.1983
531,43.8010,87.6005
572,38.5000,38.0000


In [50]:
# Add marker layer ontop of heat map

fig.add_layer(heat_layer)


marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Display figure
fig




Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…